In [ ]:
from astropy.io import fits
import numpy as np
from scipy.ndimage import interpolation as interp
import glob

# Download FITS data

In [ ]:
%%bash
mkdir fits_data

for FITSSITE in https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f435w_v1_sci.fits https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f606w_v1_sci.fits https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f775w_v1_sci.fits https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f814w_v1_sci.fits https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f850lp_v1_sci.fits
do
    echo $FITSSITE
    wget -P fits_data $FITSSITE
done

# Preprocess FITS file

In [ ]:
def rotn(xdf):
    """
    Rotate and crop FITS files.
    """
    xdf = interp.rotate(xdf, 43.25)
    xdf = xdf[2080:5320,2080:5320]
    return xdf

def min_max_norm(ar):
    return (ar - ar.min())/(ar.max() - ar.min())

xdfs = [fits.open(ffile)[0].data for ffile in np.sort(glob.glob("./fits_data/*.fits"))]
rot_xdf = [rotn(xdf) for xdf in xdfs]

clipped_xdf = []
for channel in rot_xdf:
    print("Clipping...")
    print("Shape:", channel.shape)
    print("Max:", np.abs(channel.max()))
    cp = np.percentile(channel, [99.99])
    channel[channel > cp[0]] = cp[0]
    print("Clipped max:", np.abs(channel.max()))
    
    print("Total before:", channel.max(), channel.min())
    channel = min_max_norm(channel)
    print("Total after:", channel.max(), channel.min())
    
    clipped_xdf.append(channel)
    
clipped_xdf = np.swapaxes(clipped_xdf, 0, -1)
np.save("./fits_data/mc_channelwise_clipping.npy", clipped_xdf)